# Regras de associação

Este notebook tem como objetivo demonstar a implementação de um algoritmo que extrai regras de associação existentes em um classico exemplo de um carrinho de compras.

## Caso

Encontrar uma relação de venda entre produtos de um estabelecimento, onde, sempre que um produto $A$ for adquirido um produto $B$ é adquirido junto.

### Vendas efetuadas

Vamos simular as vendas de determinados produtos em um estabelecimento

* Venda 1 - Acuçar, Leite, Pão 

* Venda 2 - Acuçar, Café, Leite, Pão 

* Venda 3 - Café, Chocolate em Pó, Leite

* Venda 4 - Chocolate em Pó, Leite, Pão

* Venda 5 - Café, Pão

* Venda 6 - Biscoito, Leite

* Venda 7 - Leite, Pão

* Venda 8 - Biscoito, Leite, Pão

* Venda 9 - Acuçar, Chocolate em Pó, Leite

* Venda 10 - Acuçar, Biscoito, Café


In [2]:
vendas = [['Acuçar', 'Leite', 'Pão'],
          ['Acuçar', 'Café', 'Leite', 'Pão'],
          ['Café', 'Chocolate em Pó', 'Leite'],
          ['Chocolate em Pó', 'Leite', 'Pão'],
          ['Café', 'Pão'],
          ['Biscoito', 'Leite'],
          ['Leite', 'Pão'],
          ['Biscoito', 'Leite', 'Pão'],
          ['Acuçar', 'Chocolate em Pó', 'Leite'],
          ['Acuçar', 'Biscoito', 'Café']]

print(vendas)

[['Acuçar', 'Leite', 'Pão'], ['Acuçar', 'Café', 'Leite', 'Pão'], ['Café', 'Chocolate em Pó', 'Leite'], ['Chocolate em Pó', 'Leite', 'Pão'], ['Café', 'Pão'], ['Biscoito', 'Leite'], ['Leite', 'Pão'], ['Biscoito', 'Leite', 'Pão'], ['Acuçar', 'Chocolate em Pó', 'Leite'], ['Acuçar', 'Biscoito', 'Café']]


## Vamos definir alguns conceitos

Para podemos minerar as regras de associação dentro dessas transações de vendas, primeiramente precisamos nos familiarizar com alguns conceitos utilizados por esses algoritmos.

### Suporte

Suporte é um indicador percentual de quantas vezes uma determinada relação, simbolizada por $A \xrightarrow[]{} B$, acontece.

Ex: Quem compra $Leite$ também compra $Pão$

Este indicador é calculado pela seguinte formula:

$sup(A \xrightarrow[]{} B) = \frac{\ número \ de \ transações \ com \ A \ e \ B}{\ número \ total \ de \ transações}$

Indo para o nosso exemplo:

$sup(A \xrightarrow[]{} B) = \frac{Quem \ compra \ Leite \  também \ compra \ Pão}{Vendas} = \frac{5}{10}$ = 0.5

### Confiança

Confiança é o grau de certeza de uma regra  $A \xrightarrow[]{} B$ acontecer com base no seu predecessor (nesse caso $A$).

$conf(A \xrightarrow[]{} B) = \frac{número \ de \ transações \ que \ suportam \  A \ \cup \ B}{\ número \ de \ transações \ que \ suportam \ A}$

Indo para o nosso exemplo:

$conf(A \xrightarrow[]{} B) = \frac{Quem \ compra \ Leite \  também \ compra \ Pão}{Quem \ compra \ Leite} = \frac{5}{8} = 0.625$

## Aplicando o algoritmo

Vamos aplicar agora o algoritmo para converter essas transações em uma matriz com valores binarios, 0 ou 1, atrelando cada produto a uma coluna. Se um item for comprado o valor atribuido será 1 caso não tenha sido, será 0.

In [5]:
# Importando pacotes para fazer o processamento
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Convertendo as transações em uma matriz com valores binarios
encoder = TransactionEncoder()
vendas_binaria = encoder.fit_transform(vendas)

df = pd.DataFrame(vendas_binaria, columns=encoder.columns_)
df

,Acuçar,Biscoito,Café,Chocolate em Pó,Leite,Pão
0,True,False,False,False,True,True
1,True,False,True,False,True,True
2,False,False,True,True,True,False
3,False,False,False,True,True,True
4,False,False,True,False,False,True
5,False,True,False,False,True,False
6,False,False,False,False,True,True
7,False,True,False,False,True,True
8,True,False,False,True,True,False
9,True,True,True,False,False,False


In [14]:
# Vamos extrair os itens frequentes. Vamos definir um valor de suporte com percentual de 20 %
itens_frequentes = apriori(df, min_support=0.2, use_colnames=True)
itens_frequentes

,support,itemsets
0,0.4,(Acuçar)
1,0.3,(Biscoito)
2,0.4,(Café)
3,0.3,(Chocolate em Pó)
4,0.8,(Leite)
5,0.6,(Pão)
6,0.2,"(Acuçar, Café)"
7,0.3,"(Acuçar, Leite)"
8,0.2,"(Acuçar, Pão)"
9,0.2,"(Biscoito, Leite)"


In [15]:
# Vamos agora extrair as regras com base nos itens frequentes obtidos utilizando a confiança com valor de 70%
regras = association_rules(itens_frequentes, metric='confidence', min_threshold=0.7)
regras

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Acuçar),(Leite),0.4,0.8,0.3,0.750000,0.937500,-0.02,0.8
1,(Chocolate em Pó),(Leite),0.3,0.8,0.3,1.000000,1.250000,0.06,inf
2,(Pão),(Leite),0.6,0.8,0.5,0.833333,1.041667,0.02,1.2
3,"(Acuçar, Pão)",(Leite),0.2,0.8,0.2,1.000000,1.250000,0.04,inf


## Analisando o resultado

Com base nos parametros de *suporte* e *confiança* conseguimos obter regras que poderão ser utilizadas para possiveis promoções ou outras ações. Existem diferentes tipos de metricas que podem ser utilizadas para melhorar o resultado e obter regras mais refinadas. 

Com esse exemplo fica claro observar que um algoritmo dessa familia pode nos ajudar a obter insigth valiosos e que agrega ainda mais valor ao negocio.